### Using Python version 3.6 and Tensorflow version 1.7

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
tf.set_random_seed(10)
import time

/Users/meenakshiparyani/anaconda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### 1. (10pts) Define functions

In [2]:
# One hot encode the labels
def one_hot_encode(y):
    targets = np.array(np.unique(y)).reshape(-1)
    enc = np.eye(len(targets))[y]
    return enc

#Get the accuracy of the results
def get_accuracy(y, y_pred):
    y = tf.nn.softmax(y)
    y_pred = tf.nn.softmax(y_pred)
    labels_equal = tf.equal(y, y_pred)
    accuracy = tf.reduce_mean(tf.cast(labels_equal, 'float'))
    return accuracy

# Get the cost of iteration
def get_cost(y, y_pred):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred, labels=y))
    return cost

def plotCostHistory(cost_history):
     plt.plot(cost_history)
     plt.ylabel('Cost');
     plt.xlabel('Iterations');
     plt.title('Cost Progression with Iterations')
     plt.show()

def extract_parameters(hyper_params, key):
    hyper_param = hyper_params[key]
    pad = hyper_param["pad"]
    f = hyper_param["f"]
    stride = hyper_param["stride"]
    filters = hyper_param["channels"]
    #a_prev = tf.pad(a_prev, [[0, 0], [pad, pad], [pad, pad], [0, 0]], "CONSTANT")
    return pad, f, stride, filters

# Flatten the array
def flatten(A):
    return tf.layers.flatten(A)

# Forward propagation to calculate yHat
def forward_propagate_lenet5(x, hyper_params):
    print("************In forward************")
    # Layer 1 Convolution 
    pad_c1, f_c1, stride_c1, filters_out_c1 =  extract_parameters(hyper_params, "c1")
    a1_c = tf.layers.conv2d(inputs=x, filters=filters_out_c1, kernel_size=[f_c1,f_c1],strides=stride_c1, 
                            padding='same',activation=tf.nn.relu) # Relu Activation
    
    print(a1_c)
    
    # Layer 1 Pooling 
    pad_p1, f_p1, stride_p1, filters_out_p1 =  extract_parameters(hyper_params, "p1")
    print(f_p1)
    a1_p = tf.layers.max_pooling2d(inputs = a1_c,pool_size=[f_p1,f_p1], strides=stride_p1)
    print(a1_p)
 
    # Layer 2 Convolution 
    
    pad_c2, f_c2, stride_c2, filters_out_c2 =  extract_parameters(hyper_params, "c2")
    print(" pad {} f {} stid {} filters {}".format(pad_c2, f_c2, stride_c2, filters_out_c2))
    a2_c = tf.layers.conv2d(inputs = a1_p, filters=filters_out_c2, kernel_size=[f_c2,f_c2]
                    ,strides=stride_c2, padding='valid' , activation=tf.nn.relu) # Relu Activation
    print(a2_c)
    # Layer 2 Pooling 
    pad_p2, f_p2, stride_p2, filters_out_p2 =  extract_parameters(hyper_params, "p2")
    a2_p = tf.layers.max_pooling2d(inputs = a2_c, pool_size=[f_p2,f_p2], strides=stride_p2)
    
    print(a2_p)
    # Flatten the array
    a3 = flatten(a2_p)
    print(a3)
    

    # Fully Connected Layer 4
    fc1_size = hyper_params["4"]["size"]
    he_init = tf.contrib.layers.xavier_initializer()
    random_init = tf.random_uniform_initializer()
    a4 = tf.layers.dense(inputs = a3, units=fc1_size, activation=tf.nn.relu, kernel_initializer=he_init)
    print(a4)
    
    # Fully Connected Layer 5
    fc2_size = hyper_params["5"]["size"]
    xavier_init = tf.contrib.layers.xavier_initializer()
    random_init = tf.random_uniform_initializer()
    a5 = tf.layers.dense(inputs = a4, units=fc2_size, activation=tf.nn.sigmoid, kernel_initializer=xavier_init)
    print(a5)
    return a5

### 2. Load data

In [3]:
# Load Training Data
X_train = np.load('exam2_train_x.npy')
y_train = np.load('exam2_train_y.npy')
# indx = 2
# plt.imshow(X_train[indx])
# plt.show()
# print("Digit in the image is " + str(y_train[indx]))

# Normalize the training data
X_train_norm = (X_train/255.0) - 0.5
y_train_encoded = one_hot_encode(y_train)

# Load testing Data
X_test = np.load('exam2_test_x.npy')
y_test = np.load('exam2_test_y.npy')
# indx = 2
# plt.imshow(X_test[indx])
# plt.show()
# print("Digit in the image is " + str(y_test[indx]))

# Normalize the test data
X_test_norm = (X_test/255.0) - 0.5
y_test_encoded = one_hot_encode(y_test)
print(X_test_norm.shape)
print(X_train_norm.shape)

print(y_train_encoded.shape)
print(y_test_encoded.shape)

(180, 64, 64, 3)
(1020, 64, 64, 3)
(1020, 6)
(180, 6)


### 3. (10pts) Initialize parameters (Weights, bias for each layer)

In [4]:
def initialize_hyper_params():
    
    hyper_params = {}
    
    # Convolution Layer 1 Hyper Parameters
    hyper_params_c1 = {}
    hyper_params_c1["stride"] = 2
    hyper_params_c1["pad"] = 1
    hyper_params_c1["f"] = 2
    hyper_params_c1["channels"] = 8
    
    # Pooling Layer 1 Hyper Parameters
    hyper_params_p1 = {}
    hyper_params_p1["stride"] = 1
    hyper_params_p1["pad"] = 0
    hyper_params_p1["f"] = 2
    hyper_params_p1["channels"] = 8
    
    # Convolution Layer 2 Hyper Parameters
    hyper_params_c2 = {}
    hyper_params_c2["stride"] = 2
    hyper_params_c2["pad"] = 0
    hyper_params_c2["f"] = 2
    hyper_params_c2["channels"] = 16
    
    # Pooling Layer 2 Hyper Parameters
    hyper_params_p2 = {}
    hyper_params_p2["stride"] = 1
    hyper_params_p2["pad"] = 0
    hyper_params_p2["f"] = 2
    hyper_params_p2["channels"] = 16
    
    hyper_params_4 = {}
    hyper_params_4["size"] = 108
            
    hyper_params_5 = {}
    hyper_params_5["size"] = 6
    
    hyper_params["c1"] = hyper_params_c1
    hyper_params["p1"] = hyper_params_p1
    
    hyper_params["c2"] = hyper_params_c2
    hyper_params["p2"] = hyper_params_p2
    
    hyper_params["4"] = hyper_params_4
    hyper_params["5"] = hyper_params_5
    
    return hyper_params

In [5]:
# hyper_params = initialize_hyper_params()
# (m, nH, nW, nC) = X_train_norm.shape
# X_train_norm = tf.placeholder(tf.float32, [None, nH, nW, nC])
# print(X_train_norm.shape)
# y_pred = forward_propagate_lenet5(X_train_norm, hyper_params)

In [6]:
# hyper_params = initialize_hyper_params()
# (m, nH, nW, nC) = X_train_norm.shape
# x = tf.placeholder(tf.float32, [None, nH, nW, nC])
# y = tf.placeholder(tf.float32)
# y_pred = tf.placeholder(tf.float32)
# y_pred = forward_propagate_lenet5(x, hyper_params)
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred, labels=y))

# optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

# init = tf.global_variables_initializer()

# with tf.Session() as sess:
#     sess.run(init)
#     for epoch in range(1000):
#         _, c = sess.run([optimizer, cost], feed_dict = {x: X_train_norm, y: y_train_encoded})
#         if epoch % 10 == 0:
#             print ("Epoch:", (epoch), "cost =", "{:.5f}".format(c))
#             #print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)) 
#     actual_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
#     accuracy = tf.reduce_mean(tf.cast(actual_prediction, tf.float32))
#     accuracy_tr = accuracy.eval({x: X_train_norm, y: y_train_encoded})
#     accuracy_te = accuracy.eval({x: X_test_norm, y: y_test_encoded})
#     print ("Accuracy on Train Dataset using keep_prob = " + " :", accuracy_tr)
#     print("Accuracy on Test Dataset using keep_prob = "+ " :", accuracy_te)
# elapsed_time = time.time() - start_time
# print ("Time taken to run: ", float(elapsed_time/60))

### 4. (40pts) Build Convolution Neural Network model

In [9]:
def train_cnn_lenet5(epochs, X_train, y_train, X_test, y_test, learning_rate, keep_prob):
    tf.reset_default_graph()
    (m, nH, nW, nC) = X_train.shape
    x = tf.placeholder(tf.float32, [None, nH, nW, nC])
    y = tf.placeholder(tf.float32)
    hyper_params = initialize_hyper_params()
    y_pred = forward_propagate_lenet5(x, hyper_params)
    cost = get_cost(y, y_pred)
    
    # The AdamOptimizer is used to optimize the cost
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) 
    init = tf.global_variables_initializer()
    correctness = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1)) # Softmax Predictions
    accuracy = tf.reduce_mean(tf.cast(correctness, 'float'))
    cost_history = []
    max_test_acc = 0
    max_iter = 0
    train_acc = 0
    test_acc = 0
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(epochs+1):
            temp, epoch_cost = sess.run([optimizer, cost], feed_dict={x:X_train, y:y_train})
            cost_history.append(epoch_cost)
            if(epoch % 1 == 0):
                train_acc = accuracy.eval({x:X_train, y:y_train})
                test_acc = accuracy.eval({x:X_test, y:y_test}) 
                if(test_acc > max_test_acc):
                    max_test_acc = test_acc
                    print('Max till now is ' , max_test_acc)
                    max_iter = epoch
                print('Accuracy Test: ' + str(test_acc) + ' on iteration ' + str(epoch))
                print('Cost is ' + str(epoch_cost) + ' for iteration: ' + str(epoch))
        plotCostHistory(cost_history)
        train_acc = train_acc * 100.0
        test_acc = test_acc * 100.0
        print('Train Accuracy: ', train_acc)
        print('Test Accuracy: ' ,test_acc)
        print('Max Test Accuracy: ' + str(max_test_acc* 100.0) + ' on iteration ' + str(max_iter))
        return train_acc, test_acc

### 5. (20pts) Optimization of Convolution Neural Network model

In [10]:
train_cnn_lenet5(1000, X_train_norm, y_train_encoded, X_test_norm, y_test_encoded, 0.01, 0.8)

************In forward************
Tensor("conv2d/Relu:0", shape=(?, 32, 32, 8), dtype=float32)
2
Tensor("max_pooling2d/MaxPool:0", shape=(?, 31, 31, 8), dtype=float32)
 pad 0 f 2 stid 2 filters 16
Tensor("conv2d_1/Relu:0", shape=(?, 15, 15, 16), dtype=float32)
Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 14, 14, 16), dtype=float32)
Tensor("flatten/Reshape:0", shape=(?, 3136), dtype=float32)
Instructions for updating:
Use the retry module or similar alternatives.
Tensor("dense/Relu:0", shape=(?, 108), dtype=float32)
Tensor("dense_1/Sigmoid:0", shape=(?, 6), dtype=float32)
Max till now is  0.22222222
Accuracy Test: 0.22222222 on iteration 0
Cost is 1.7921537 for iteration: 0
Accuracy Test: 0.16666667 on iteration 1
Cost is 1.7691069 for iteration: 1
Accuracy Test: 0.15555556 on iteration 2
Cost is 1.7926015 for iteration: 2
Max till now is  0.30555555
Accuracy Test: 0.30555555 on iteration 3
Cost is 1.7525957 for iteration: 3
Max till now is  0.4888889
Accuracy Test: 0.4888889 on itera

Accuracy Test: 0.82222223 on iteration 89
Cost is 1.0742517 for iteration: 89
Accuracy Test: 0.8333333 on iteration 90
Cost is 1.0739588 for iteration: 90
Max till now is  0.84444445
Accuracy Test: 0.84444445 on iteration 91
Cost is 1.0732982 for iteration: 91
Accuracy Test: 0.84444445 on iteration 92
Cost is 1.073019 for iteration: 92
Accuracy Test: 0.8388889 on iteration 93
Cost is 1.0727459 for iteration: 93
Accuracy Test: 0.8333333 on iteration 94
Cost is 1.0724217 for iteration: 94
Accuracy Test: 0.8277778 on iteration 95
Cost is 1.0721692 for iteration: 95
Accuracy Test: 0.8333333 on iteration 96
Cost is 1.0719553 for iteration: 96
Accuracy Test: 0.8388889 on iteration 97
Cost is 1.0716062 for iteration: 97
Accuracy Test: 0.84444445 on iteration 98
Cost is 1.0713987 for iteration: 98
Accuracy Test: 0.84444445 on iteration 99
Cost is 1.0711364 for iteration: 99
Accuracy Test: 0.84444445 on iteration 100
Cost is 1.0708534 for iteration: 100
Accuracy Test: 0.8333333 on iteration 101

Accuracy Test: 0.8611111 on iteration 193
Cost is 1.0599601 for iteration: 193
Accuracy Test: 0.8611111 on iteration 194
Cost is 1.0599405 for iteration: 194
Accuracy Test: 0.8611111 on iteration 195
Cost is 1.0599235 for iteration: 195
Accuracy Test: 0.8611111 on iteration 196
Cost is 1.059907 for iteration: 196
Accuracy Test: 0.8666667 on iteration 197
Cost is 1.0598925 for iteration: 197
Accuracy Test: 0.8666667 on iteration 198
Cost is 1.0598818 for iteration: 198
Accuracy Test: 0.8666667 on iteration 199
Cost is 1.0598743 for iteration: 199
Accuracy Test: 0.85555553 on iteration 200
Cost is 1.0598677 for iteration: 200
Accuracy Test: 0.85 on iteration 201
Cost is 1.0598608 for iteration: 201
Accuracy Test: 0.85 on iteration 202
Cost is 1.0598538 for iteration: 202
Accuracy Test: 0.85 on iteration 203
Cost is 1.0598477 for iteration: 203
Accuracy Test: 0.85 on iteration 204
Cost is 1.0598426 for iteration: 204
Accuracy Test: 0.85 on iteration 205
Cost is 1.0598383 for iteration: 20

Accuracy Test: 0.85 on iteration 297
Cost is 1.0588024 for iteration: 297
Accuracy Test: 0.85 on iteration 298
Cost is 1.0587816 for iteration: 298
Accuracy Test: 0.85555553 on iteration 299
Cost is 1.0587369 for iteration: 299
Accuracy Test: 0.8666667 on iteration 300
Cost is 1.0587326 for iteration: 300
Accuracy Test: 0.85555553 on iteration 301
Cost is 1.0586692 for iteration: 301
Accuracy Test: 0.85555553 on iteration 302
Cost is 1.0586367 for iteration: 302
Accuracy Test: 0.85555553 on iteration 303
Cost is 1.0586402 for iteration: 303
Accuracy Test: 0.85555553 on iteration 304
Cost is 1.0586255 for iteration: 304
Accuracy Test: 0.8611111 on iteration 305
Cost is 1.0585741 for iteration: 305
Accuracy Test: 0.8666667 on iteration 306
Cost is 1.0584875 for iteration: 306
Accuracy Test: 0.8611111 on iteration 307
Cost is 1.0585288 for iteration: 307
Accuracy Test: 0.85555553 on iteration 308
Cost is 1.0584494 for iteration: 308
Accuracy Test: 0.85 on iteration 309
Cost is 1.0584366 f

Accuracy Test: 0.8611111 on iteration 401
Cost is 1.0575937 for iteration: 401
Accuracy Test: 0.8611111 on iteration 402
Cost is 1.057593 for iteration: 402
Accuracy Test: 0.8611111 on iteration 403
Cost is 1.0575923 for iteration: 403
Accuracy Test: 0.8611111 on iteration 404
Cost is 1.0575913 for iteration: 404
Accuracy Test: 0.8611111 on iteration 405
Cost is 1.0575904 for iteration: 405
Accuracy Test: 0.8611111 on iteration 406
Cost is 1.0575897 for iteration: 406
Accuracy Test: 0.8611111 on iteration 407
Cost is 1.057589 for iteration: 407
Accuracy Test: 0.8611111 on iteration 408
Cost is 1.0575886 for iteration: 408
Accuracy Test: 0.8611111 on iteration 409
Cost is 1.057588 for iteration: 409
Accuracy Test: 0.8611111 on iteration 410
Cost is 1.0575871 for iteration: 410
Accuracy Test: 0.8611111 on iteration 411
Cost is 1.0575866 for iteration: 411
Accuracy Test: 0.8666667 on iteration 412
Cost is 1.0575861 for iteration: 412
Accuracy Test: 0.8666667 on iteration 413
Cost is 1.057

...

Accuracy Test: 0.8666667 on iteration 609
Cost is 1.0575644 for iteration: 609
Accuracy Test: 0.8666667 on iteration 610
Cost is 1.0575641 for iteration: 610
Accuracy Test: 0.8666667 on iteration 611
Cost is 1.0575641 for iteration: 611
Accuracy Test: 0.8666667 on iteration 612
Cost is 1.0575641 for iteration: 612
Accuracy Test: 0.8666667 on iteration 613
Cost is 1.0575641 for iteration: 613
Accuracy Test: 0.8666667 on iteration 614
Cost is 1.0575641 for iteration: 614
Accuracy Test: 0.8666667 on iteration 615
Cost is 1.0575639 for iteration: 615
Accuracy Test: 0.8666667 on iteration 616
Cost is 1.0575639 for iteration: 616
Accuracy Test: 0.8666667 on iteration 617
Cost is 1.0575639 for iteration: 617
Accuracy Test: 0.8666667 on iteration 618
Cost is 1.0575639 for iteration: 618
Accuracy Test: 0.8666667 on iteration 619
Cost is 1.0575638 for iteration: 619
Accuracy Test: 0.8666667 on iteration 620
Cost is 1.0575637 for iteration: 620
Accuracy Test: 0.8666667 on iteration 621
Cost is 1.

Accuracy Test: 0.8666667 on iteration 713
Cost is 1.0575609 for iteration: 713
Accuracy Test: 0.8666667 on iteration 714
Cost is 1.0575607 for iteration: 714
Accuracy Test: 0.8666667 on iteration 715
Cost is 1.0575607 for iteration: 715
Accuracy Test: 0.8666667 on iteration 716
Cost is 1.0575607 for iteration: 716
Accuracy Test: 0.8666667 on iteration 717
Cost is 1.0575607 for iteration: 717
Accuracy Test: 0.8666667 on iteration 718
Cost is 1.0575607 for iteration: 718
Accuracy Test: 0.8666667 on iteration 719
Cost is 1.0575607 for iteration: 719
Accuracy Test: 0.8666667 on iteration 720
Cost is 1.0575607 for iteration: 720
Accuracy Test: 0.8666667 on iteration 721
Cost is 1.0575606 for iteration: 721
Accuracy Test: 0.8666667 on iteration 722
Cost is 1.0575606 for iteration: 722
Accuracy Test: 0.8666667 on iteration 723
Cost is 1.0575606 for iteration: 723
Accuracy Test: 0.8666667 on iteration 724
Cost is 1.0575606 for iteration: 724
Accuracy Test: 0.8666667 on iteration 725
Cost is 1.

Accuracy Test: 0.8666667 on iteration 818
Cost is 1.0575585 for iteration: 818
Accuracy Test: 0.8666667 on iteration 819
Cost is 1.0575585 for iteration: 819
Accuracy Test: 0.8666667 on iteration 820
Cost is 1.0575585 for iteration: 820
Accuracy Test: 0.8666667 on iteration 821
Cost is 1.0575585 for iteration: 821
Accuracy Test: 0.8666667 on iteration 822
Cost is 1.0575585 for iteration: 822
Accuracy Test: 0.8666667 on iteration 823
Cost is 1.0575585 for iteration: 823
Accuracy Test: 0.8666667 on iteration 824
Cost is 1.0575585 for iteration: 824
Accuracy Test: 0.8666667 on iteration 825
Cost is 1.0575584 for iteration: 825
Accuracy Test: 0.8666667 on iteration 826
Cost is 1.0575584 for iteration: 826
Accuracy Test: 0.8666667 on iteration 827
Cost is 1.0575584 for iteration: 827
Accuracy Test: 0.8666667 on iteration 828
Cost is 1.0575584 for iteration: 828
Accuracy Test: 0.8666667 on iteration 829
Cost is 1.0575584 for iteration: 829
Accuracy Test: 0.8666667 on iteration 830
Cost is 1.

Accuracy Test: 0.8666667 on iteration 922
Cost is 1.0575569 for iteration: 922
Accuracy Test: 0.8666667 on iteration 923
Cost is 1.0575569 for iteration: 923
Accuracy Test: 0.8666667 on iteration 924
Cost is 1.0575569 for iteration: 924
Accuracy Test: 0.8666667 on iteration 925
Cost is 1.0575569 for iteration: 925
Accuracy Test: 0.8666667 on iteration 926
Cost is 1.0575567 for iteration: 926
Accuracy Test: 0.8666667 on iteration 927
Cost is 1.0575566 for iteration: 927
Accuracy Test: 0.8666667 on iteration 928
Cost is 1.0575565 for iteration: 928
Accuracy Test: 0.8666667 on iteration 929
Cost is 1.0575564 for iteration: 929
Accuracy Test: 0.8666667 on iteration 930
Cost is 1.0575562 for iteration: 930
Accuracy Test: 0.8666667 on iteration 931
Cost is 1.0575556 for iteration: 931
Accuracy Test: 0.8666667 on iteration 932
Cost is 1.0575542 for iteration: 932
Accuracy Test: 0.87222224 on iteration 933
Cost is 1.0575496 for iteration: 933
Accuracy Test: 0.8666667 on iteration 934
Cost is 1

NameError: name 'plotCostHistory' is not defined

### 5. (20pts) Predictions